In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(keras.__version__)

2.8.2
2.8.0


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

dataset = pd.read_csv('/content/drive/MyDrive/datasets/movie.csv', sep=",")
X = dataset["text"]
y = dataset["label"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=True, stratify=y)

X_train, y_train

(23471    This is definitely one of the better Mel Brook...
 819      After playing a nymphomaniac in WRITTEN ON THE...
 5848     I've seen a lot of movies in my time and this ...
 36330    This is an example of why the majority of acti...
 36497    By watching this film you will not only explor...
                                ...                        
 17159    I never much liked the Myra movie, tho I appre...
 20379    Committed stars (Heather Graham) along with (C...
 7540     Well...I like Patricia Kaas. She is a beautifu...
 39557    Dull acting, weak script...worst spanish movie...
 1583     The only possible way to enjoy this flick is t...
 Name: text, Length: 20000, dtype: object, 23471    1
 819      1
 5848     0
 36330    0
 36497    1
         ..
 17159    0
 20379    0
 7540     0
 39557    0
 1583     0
 Name: label, Length: 20000, dtype: int64)

## Ensenble 
### Bidirectional GRU 32
### SVM

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

vectorizer = TfidfVectorizer(stop_words='english')

X_train_sv = vectorizer.fit_transform(X_train)
clf1 = SVC(C=10, gamma=0.1, kernel="rbf", verbose=True)

clf1.fit(X_train_sv, y_train)


[LibSVM]

In [ ]:
from keras.models import Sequential
from keras.callbacks import History, EarlyStopping
from keras.layers import Embedding, TextVectorization, Dense, LSTM, Input, Bidirectional

vectorize_layer = TextVectorization(max_tokens=50000)
vectorize_layer.adapt(X_train)

history_Adam = History()

model = Sequential()

# Vectorization
model.add(Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)

# Embedding
model.add(Embedding(input_dim=len(vectorize_layer.get_vocabulary()), output_dim=64))


model.add(Bidirectional(tf.keras.layers.GRU(32)))
model.add(Dense(24, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


model.summary()


In [ ]:
model.compile(loss="binary_crossentropy",optimizer="adam", metrics=["accuracy"])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, mode='min', verbose=1)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=100, callbacks=[history_Adam, early_stopping])

In [ ]:
from mlxtend.classifier import EnsembleVoteClassifier
eclf = EnsembleVoteClassifier(clfs=[clf1, model], weights=[1,1], fit_base_estimators=False)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

for name, score in [("accuracy", accuracy_score), ("recall", recall_score), ("precision", precision_score), ("f1", f1_score)]:
    print(name, score(grid_search.predict(X_test), y_test))
